In [1]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
 
# Create a new circuit with two qubits
qc = QuantumCircuit(2)
 
# Add a Hadamard gate to qubit 0
qc.h(0)
 
# Perform a controlled-X gate on qubit 1, controlled by qubit 0
qc.cx(0, 1)
 
# Return a drawing of the circuit using MatPlotLib ("mpl"). This is the
# last line of the cell, so the drawing appears in the cell output.
# Remove the "mpl" argument to get a text drawing.
qc.draw()


┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

In [64]:
for inst in qc.data:
    print(inst)

CircuitInstruction(operation=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0),), clbits=())
CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)), clbits=())


In [2]:
# Set up six different observables.
from qiskit.quantum_info import SparsePauliOp
 
observables_labels = ["IZ", "IX", "ZI", "XI", "ZZ", "XX"]
observables = [SparsePauliOp(label) for label in observables_labels]


In [4]:
import qiskit_aer
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime.fake_provider import FakeVigo
from qiskit import transpile


In [65]:
backend = qiskit_aer.AerSimulator.from_backend(FakeVigo())
 
# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)
 
isa_circuit.draw()

global phase: π/4
               ┌─────────┐┌────┐┌─────────┐     
      q_0 -> 0 ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├──■──
               └─────────┘└────┘└─────────┘┌─┴─┐
      q_1 -> 1 ────────────────────────────┤ X ├
                                           └───┘
ancilla_0 -> 2 ─────────────────────────────────
                                                
ancilla_1 -> 3 ─────────────────────────────────
                                                
ancilla_2 -> 4 ─────────────────────────────────

In [83]:
for qubit in isa_circuit.qubits:
    print(qubit._register)

QuantumRegister(5, 'q')
QuantumRegister(5, 'q')
QuantumRegister(5, 'q')
QuantumRegister(5, 'q')
QuantumRegister(5, 'q')


In [86]:
isa_circuit.layout

TranspileLayout(initial_layout=Layout({
0: Qubit(QuantumRegister(2, 'q'), 0),
1: Qubit(QuantumRegister(2, 'q'), 1),
2: Qubit(QuantumRegister(3, 'ancilla'), 0),
3: Qubit(QuantumRegister(3, 'ancilla'), 1),
4: Qubit(QuantumRegister(3, 'ancilla'), 2)
}), input_qubit_mapping={Qubit(QuantumRegister(2, 'q'), 0): 0, Qubit(QuantumRegister(2, 'q'), 1): 1, Qubit(QuantumRegister(3, 'ancilla'), 0): 2, Qubit(QuantumRegister(3, 'ancilla'), 1): 3, Qubit(QuantumRegister(3, 'ancilla'), 2): 4}, final_layout=None, _input_qubit_count=2, _output_qubit_list=[Qubit(QuantumRegister(5, 'q'), 0), Qubit(QuantumRegister(5, 'q'), 1), Qubit(QuantumRegister(5, 'q'), 2), Qubit(QuantumRegister(5, 'q'), 3), Qubit(QuantumRegister(5, 'q'), 4)])

In [89]:
isa_circuit.layout

TranspileLayout(initial_layout=Layout({
0: Qubit(QuantumRegister(2, 'q'), 0),
1: Qubit(QuantumRegister(2, 'q'), 1),
2: Qubit(QuantumRegister(3, 'ancilla'), 0),
3: Qubit(QuantumRegister(3, 'ancilla'), 1),
4: Qubit(QuantumRegister(3, 'ancilla'), 2)
}), input_qubit_mapping={Qubit(QuantumRegister(2, 'q'), 0): 0, Qubit(QuantumRegister(2, 'q'), 1): 1, Qubit(QuantumRegister(3, 'ancilla'), 0): 2, Qubit(QuantumRegister(3, 'ancilla'), 1): 3, Qubit(QuantumRegister(3, 'ancilla'), 2): 4}, final_layout=None, _input_qubit_count=2, _output_qubit_list=[Qubit(QuantumRegister(5, 'q'), 0), Qubit(QuantumRegister(5, 'q'), 1), Qubit(QuantumRegister(5, 'q'), 2), Qubit(QuantumRegister(5, 'q'), 3), Qubit(QuantumRegister(5, 'q'), 4)])

In [91]:
isa_circuit.duration

In [ ]:
input_qubit_mapping={
    Qubit(QuantumRegister(2, 'q'), 0): 0, 
    Qubit(QuantumRegister(2, 'q'), 1): 1, 
    Qubit(QuantumRegister(3, 'ancilla'), 0): 2, 
    Qubit(QuantumRegister(3, 'ancilla'), 1): 3,
    Qubit(QuantumRegister(3, 'ancilla'), 2): 4
}, 
final_layout=None, 
_input_qubit_count=2, 
_output_qubit_list=[
    Qubit(QuantumRegister(5, 'q'), 0), 
    Qubit(QuantumRegister(5, 'q'), 1), 
    Qubit(QuantumRegister(5, 'q'), 2), 
    Qubit(QuantumRegister(5, 'q'), 3), 
    Qubit(QuantumRegister(5, 'q'), 4)
]




In [6]:
# Construct the Estimator instance.
from qiskit_ibm_runtime import EstimatorV2 as Estimator
 
estimator = Estimator(backend=backend)
estimator.options.resilience_level = 1
estimator.options.default_shots = 5000
 
mapped_observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]
 
# One pub, with one circuit to run against five different observables.
job = estimator.run([(isa_circuit, mapped_observables)])

/Users/hyeokkim/opt/miniconda3/envs/patoka/lib/python3.12/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:243: UserWarning: Options {'default_shots': 5000} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


In [59]:
from qiskit.quantum_info import Pauli
from qiskit import QuantumCircuit, qasm3, qasm2

c = job._args[0][0].circuit.copy()
# paulis = mapped_observables
observables = job._args[0][0].observables
paulis = {pauli for obs_dict in observables.ravel() for pauli in obs_dict.keys()}
for pauli in paulis:
    c.save_expectation_value(
        Pauli(pauli), qubits=range(c.num_qubits), label=pauli
    )

[CircuitInstruction(operation=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[1.5707963267948966]), qubits=(Qubit(QuantumRegister(5, 'q'), 0),), clbits=()), CircuitInstruction(operation=Instruction(name='sx', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'q'), 0),), clbits=()), CircuitInstruction(operation=Instruction(name='rz', num_qubits=1, num_clbits=0, params=[1.5707963267948966]), qubits=(Qubit(QuantumRegister(5, 'q'), 0),), clbits=()), CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(5, 'q'), 0), Qubit(QuantumRegister(5, 'q'), 1)), clbits=()), CircuitInstruction(operation=Instruction(name='save_expval', num_qubits=5, num_clbits=0, params=[('IIIXX', (1.0, 0))]), qubits=(Qubit(QuantumRegister(5, 'q'), 0), Qubit(QuantumRegister(5, 'q'), 1), Qubit(QuantumRegister(5, 'q'), 2), Qubit(QuantumRegister(5, 'q'), 3), Qubit(QuantumRegister(5, 'q'), 4)), clbits=()), CircuitInstructio

In [95]:
c.op_start_times

AttributeError: This circuit is not scheduled. To schedule it run the circuit through one of the transpiler scheduling passes.

In [9]:
res = job.result()

In [13]:
res[0].data.evs

array([0.00195312, 0.00634766, 0.04541016, 0.04150391, 0.80517578,
       0.79003906])

In [14]:
res[0].data.stds

array([0.01562497, 0.01562469, 0.01560888, 0.01561154, 0.00926619,
       0.00957901])

In [27]:
res[0].metadata

{'target_precision': 0.015625}

In [96]:
qc_a = QuantumCircuit(4, 4)
# qc_a.initialize([1/np.sqrt(2), -1/np.sqrt(2)], 0)
# matrix = [[0, 0, 0, 1],
#         [0, 0, 1, 0],
#         [1, 0, 0, 0],
#         [0, 1, 0, 0]]
# qc_a.x(0)
# qc_a.y(0)
# qc_a.z(0)
qc_a.iswap(0, 2)
# qc_a.t(2)
# qc_a.tdg(3)
# qc_a.cx(3, 2)
# qc_a.cy(3, 2)
# qc_a.cz(3, 2)
# qc_a.cswap(3, 2, 1)
# qc_a.pauli('XX', [1, 2])
# qc_a.rz(0.2, 1)
# qc_a.mcry(0.5, [0, 1], 2, q_ancillae=[3])
# qc_a.reset([0,1,2])
# qc_a.barrier()
# qc_a.cu(0.2, 0.21, 0.34, 0.25, 0, 2)
# qc_a.unitary(matrix, [0, 1])
qc_a.measure([0, 1, 2, 3], [0, 1, 2, 3])

qasm3.dumps(qc_a)
# 1. parse
# 2. draw
# 3. filter-preserve moment

'OPENQASM 3.0;\ninclude "stdgates.inc";\ngate iswap _gate_q_0, _gate_q_1 {\n  s _gate_q_0;\n  s _gate_q_1;\n  h _gate_q_0;\n  cx _gate_q_0, _gate_q_1;\n  cx _gate_q_1, _gate_q_0;\n  h _gate_q_1;\n}\nbit[4] c;\nqubit[4] q;\niswap q[0], q[2];\nc[0] = measure q[0];\nc[1] = measure q[1];\nc[2] = measure q[2];\nc[3] = measure q[3];\n'

In [97]:
# Convert to an ISA circuit and layout-mapped observables.
pm2 = generate_preset_pass_manager(backend=backend, optimization_level=1)
qca_trans = pm2.run(qc_a)

In [101]:
qca_trans.draw()

»
ancilla_0 -> 0 ──────────────────────────────────────────────────────────»
                ┌────────┐┌────┐┌─────────┐     ┌───┐           ┌─┐      »
      q_0 -> 1 ─┤ Rz(-π) ├┤ √X ├┤ Rz(π/2) ├──■──┤ X ├───────────┤M├──────»
               ┌┴────────┤└────┘└─────────┘┌─┴─┐└─┬─┘┌─────────┐└╥┘┌────┐»
      q_2 -> 2 ┤ Rz(π/2) ├─────────────────┤ X ├──■──┤ Rz(π/2) ├─╫─┤ √X ├»
               └───┬─┬───┘                 └───┘     └─────────┘ ║ └────┘»
      q_1 -> 3 ────┤M├───────────────────────────────────────────╫───────»
                   └╥┘     ┌─┐                                   ║       »
      q_3 -> 4 ─────╫──────┤M├───────────────────────────────────╫───────»
                    ║      └╥┘                                   ║       »
          c: 4/═════╩═══════╩════════════════════════════════════╩═══════»
                    1       3                                    0       »
«                             
«ancilla_0 -> 0 ──────────────
«                             
«      q_0 -> 1 ──────────────
«               ┌─────────┐┌─┐
«      q_2 -> 2 ┤ Rz(π/2) ├┤M├
«               └─────────┘└╥┘
«      q_1 -> 3 ────────────╫─
«                           ║ 
«      q_3 -> 4 ────────────╫─
«                           ║ 
«          c: 4/════════════╩═
«                           2

In [102]:
qc_a.draw()

┌────────┐   ┌─┐   
q_0: ┤0       ├───┤M├───
     │        │┌─┐└╥┘   
q_1: ┤  Iswap ├┤M├─╫────
     │        │└╥┘ ║ ┌─┐
q_2: ┤1       ├─╫──╫─┤M├
     └──┬─┬───┘ ║  ║ └╥┘
q_3: ───┤M├─────╫──╫──╫─
        └╥┘     ║  ║  ║ 
c: 4/════╩══════╩══╩══╩═
         3      1  0  2